F1 Cost Cap SCM Analysis - Exploratory Data Analysis
=====================================================

This notebook explores the data to:
1. Visualize McLaren's trajectory vs donor pool
2. Validate pre-treatment period assumptions
3. Identify treatment effects visually
4. Explore within-season dynamics (race-level data)

Author: Tomasz Solis
Date: December 2025


# ============================================================================
# SETUP
# ============================================================================


In [1]:
import sys
from pathlib import Path

# Ensure src/ is importable
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Setup complete")


✅ Setup complete


# ============================================================================
# LOAD PROCESSED DATA
# ============================================================================


In [2]:
data_dir = Path("../data/processed")

# Load annual data
annual_df = pd.read_csv(data_dir / "annual_constructor_points.csv")
treated_df = pd.read_csv(data_dir / "treated_mclaren.csv")
donor_main_df = pd.read_csv(data_dir / "donor_pool_main.csv")
donor_robust_df = pd.read_csv(data_dir / "donor_pool_robust.csv")
placebo_sauber_df = pd.read_csv(data_dir / "placebo_sauber.csv")
placebo_haas_df = pd.read_csv(data_dir / "placebo_haas.csv")

# Load race-level data (if available)
race_path = data_dir / "race_level_points.csv"
if race_path.exists():
    race_df = pd.read_csv(race_path)
    print(f"✅ Loaded race-level data: {race_df.shape}")
else:
    race_df = None
    print("⚠️  No race-level data available")

print(f"\n✅ Data loaded")
print(f"Annual data: {annual_df.shape}")
print(f"McLaren: {treated_df.shape}")
print(f"Main donor pool: {donor_main_df.shape}")

✅ Loaded race-level data: (1690, 6)

✅ Data loaded
Annual data: (80, 5)
McLaren: (8, 6)
Main donor pool: (24, 6)


# ============================================================================
# VISUALIZATION 1: McLAREN VS DONOR POOL (FULL TRAJECTORY)
# ============================================================================


In [3]:
print("\n" + "="*80)
print("VISUALIZATION 1: McLAREN VS DONOR POOL TRAJECTORY")
print("="*80)

# Create figure
fig = go.Figure()

# Add McLaren line (treated unit)
mclaren_data = treated_df.sort_values('season')
fig.add_trace(go.Scatter(
    x=mclaren_data['season'],
    y=mclaren_data['points'],
    mode='lines+markers',
    name='McLaren (Treated)',
    line=dict(color='#FF8700', width=4),  # McLaren papaya orange
    marker=dict(size=10)
))

# Add donor pool lines
donor_teams = donor_main_df['constructorName'].unique()
colors = {
    'ALPINE': '#0090FF',
    'SAUBER': '#900000',
    'HAAS': '#FFFFFF',
    'RB': '#2B4562'
}

for team in donor_teams:
    team_data = donor_main_df[donor_main_df['constructorName'] == team].sort_values('season')
    fig.add_trace(go.Scatter(
        x=team_data['season'],
        y=team_data['points'],
        mode='lines+markers',
        name=team,
        line=dict(color=colors.get(team, '#808080'), width=2, dash='dot'),
        marker=dict(size=6)
    ))

# Add vertical line at treatment year
fig.add_vline(
    x=2021.5,
    line_dash="dash",
    line_color="red",
    annotation_text="Treatment: Cost Cap + New Regs",
    annotation_position="top"
)

# Add shaded region for pre-treatment period
fig.add_vrect(
    x0=2017, x1=2020,
    fillcolor="green",
    opacity=0.1,
    annotation_text="Pre-treatment",
    annotation_position="top left"
)

# Add shaded region for treatment period
fig.add_vrect(
    x0=2022, x1=2024,
    fillcolor="orange",
    opacity=0.1,
    annotation_text="Treatment Period",
    annotation_position="top right"
)

# Update layout
fig.update_layout(
    title="McLaren vs Donor Pool: Constructor Championship Points (2017-2024)",
    xaxis_title="Season",
    yaxis_title="Constructor Points",
    hovermode='x unified',
    height=600,
    template='plotly_white',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()
fig.write_image("../outputs/figures/01_mclaren_vs_donor_pool.png", scale=2)


VISUALIZATION 1: McLAREN VS DONOR POOL TRAJECTORY


# ============================================================================
# VISUALIZATION 2: PRE-TREATMENT TRENDS (2017-2020)
# ============================================================================


In [4]:
print("\n" + "="*80)
print("VISUALIZATION 2: PRE-TREATMENT PERIOD ANALYSIS")
print("="*80)

# Filter to pre-treatment period
pre_treatment = annual_df[
    (annual_df['season'] >= 2017) & 
    (annual_df['season'] <= 2020)
]

# Calculate pre-treatment statistics
pre_stats = pre_treatment.groupby('constructorName').agg({
    'points': ['mean', 'std', 'min', 'max']
}).round(1)

print("\nPre-treatment statistics (2017-2020):")
print(pre_stats)

# Create figure with subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        'Pre-Treatment Trajectories (2017-2020)',
        'Average Points Pre-Treatment'
    )
)

# Left plot: Pre-treatment trajectories
teams_to_plot = ['MCLAREN'] + list(donor_main_df['constructorName'].unique())

for team in teams_to_plot:
    team_data = pre_treatment[pre_treatment['constructorName'] == team].sort_values('season')
    
    is_mclaren = (team == 'MCLAREN')
    
    fig.add_trace(
        go.Scatter(
            x=team_data['season'],
            y=team_data['points'],
            mode='lines+markers',
            name=team,
            line=dict(
                color='#FF8700' if is_mclaren else colors.get(team, '#808080'),
                width=4 if is_mclaren else 2
            ),
            marker=dict(size=10 if is_mclaren else 6),
            showlegend=True
        ),
        row=1, col=1
    )

# Right plot: Average points bar chart
pre_means = pre_treatment.groupby('constructorName')['points'].mean().reset_index()
pre_means = pre_means[pre_means['constructorName'].isin(teams_to_plot)]
pre_means = pre_means.sort_values('points', ascending=True)

bar_colors = ['#FF8700' if team == 'MCLAREN' else '#808080' 
              for team in pre_means['constructorName']]

fig.add_trace(
    go.Bar(
        x=pre_means['points'],
        y=pre_means['constructorName'],
        orientation='h',
        marker_color=bar_colors,
        showlegend=False
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Season", row=1, col=1)
fig.update_yaxes(title_text="Points", row=1, col=1)
fig.update_xaxes(title_text="Average Points", row=1, col=2)

fig.update_layout(
    height=500,
    template='plotly_white',
    showlegend=True
)

fig.show()
fig.write_image("../outputs/figures/02_pre_treatment_analysis.png", scale=2)


VISUALIZATION 2: PRE-TREATMENT PERIOD ANALYSIS

Pre-treatment statistics (2017-2020):
                points                     
                  mean    std    min    max
constructorName                            
ALPINE           112.8   52.7   57.0  181.0
ASTON MARTIN     141.5   59.3   73.0  195.0
FERRARI          432.0  202.7  131.0  571.0
HAAS              42.8   38.0    3.0   93.0
MCLAREN          109.8   78.3   30.0  202.0
MERCEDES         658.8   68.1  573.0  739.0
RB                69.5   32.9   33.0  107.0
RED BULL         380.8   47.4  319.0  419.0
SAUBER            29.5   26.8    5.0   57.0
WILLIAMS          22.8   40.3    0.0   83.0


# ============================================================================
# VISUALIZATION 3: TREATMENT EFFECT (2022-2024)
# ============================================================================


In [5]:
print("\n" + "="*80)
print("VISUALIZATION 3: TREATMENT PERIOD ANALYSIS")
print("="*80)

# Filter to treatment period
treatment_period = annual_df[
    (annual_df['season'] >= 2022) & 
    (annual_df['season'] <= 2024)
]

# Calculate treatment statistics
treatment_stats = treatment_period.groupby('constructorName').agg({
    'points': ['mean', 'std', 'min', 'max']
}).round(1)

print("\nTreatment period statistics (2022-2024):")
print(treatment_stats)

# Create comparison: pre vs post
comparison_data = []

for team in teams_to_plot:
    pre_mean = pre_treatment[pre_treatment['constructorName'] == team]['points'].mean()
    post_mean = treatment_period[treatment_period['constructorName'] == team]['points'].mean()
    
    comparison_data.append({
        'Team': team,
        'Pre-Treatment (2017-2020)': pre_mean,
        'Treatment (2022-2024)': post_mean,
        'Change': post_mean - pre_mean,
        'Change %': ((post_mean / pre_mean) - 1) * 100 if pre_mean > 0 else np.nan
    })

comparison_df = pd.DataFrame(comparison_data).sort_values('Change', ascending=False)

print("\n📊 Pre-Treatment vs Treatment Comparison:")
print(comparison_df.to_string(index=False))

# Visualize the change
fig = go.Figure()

fig.add_trace(go.Bar(
    name='Pre-Treatment (2017-2020)',
    x=comparison_df['Team'],
    y=comparison_df['Pre-Treatment (2017-2020)'],
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    name='Treatment (2022-2024)',
    x=comparison_df['Team'],
    y=comparison_df['Treatment (2022-2024)'],
    marker_color='orange'
))

fig.update_layout(
    title='Average Points: Pre-Treatment vs Treatment Period',
    xaxis_title='Team',
    yaxis_title='Average Points',
    barmode='group',
    height=500,
    template='plotly_white'
)

fig.show()
fig.write_image("../outputs/figures/03_pre_vs_treatment_comparison.png", scale=2)


VISUALIZATION 3: TREATMENT PERIOD ANALYSIS

Treatment period statistics (2022-2024):
                points                     
                  mean    std    min    max
constructorName                            
ALPINE           119.3   54.0   65.0  173.0
ASTON MARTIN     143.0  120.2   55.0  280.0
FERRARI          537.3  123.8  406.0  652.0
HAAS              35.7   23.0   12.0   58.0
MCLAREN          375.7  261.4  159.0  666.0
MERCEDES         464.0   53.1  409.0  515.0
RB                35.3   10.5   25.0   46.0
RED BULL         736.0  137.0  589.0  860.0
SAUBER            25.0   26.7    4.0   55.0
WILLIAMS          17.7   10.0    8.0   28.0

📊 Pre-Treatment vs Treatment Comparison:
   Team  Pre-Treatment (2017-2020)  Treatment (2022-2024)     Change   Change %
MCLAREN                     109.75             375.666667 265.916667 242.293090
 ALPINE                     112.75             119.333333   6.583333   5.838877
 SAUBER                      29.50              25.000000  -

# ============================================================================
# VISUALIZATION 4: YEAR-OVER-YEAR CHANGES
# ============================================================================


In [6]:
print("\n" + "="*80)
print("VISUALIZATION 4: YEAR-OVER-YEAR CHANGES")
print("="*80)

# Calculate year-over-year changes
yoy_changes = []

for team in teams_to_plot:
    team_data = annual_df[annual_df['constructorName'] == team].sort_values('season')
    
    for i in range(len(team_data) - 1):
        current = team_data.iloc[i]
        next_year = team_data.iloc[i + 1]
        
        yoy_changes.append({
            'Team': team,
            'Year': f"{int(current['season'])}-{int(next_year['season'])}",
            'Change': next_year['points'] - current['points'],
            'Season_Start': int(current['season'])
        })

yoy_df = pd.DataFrame(yoy_changes)

# Focus on key transitions
key_transitions = yoy_df[yoy_df['Year'].isin(['2021-2022', '2022-2023', '2023-2024'])]

fig = px.bar(
    key_transitions,
    x='Year',
    y='Change',
    color='Team',
    barmode='group',
    title='Year-over-Year Point Changes: Key Treatment Period Transitions',
    labels={'Change': 'Point Change', 'Year': 'Transition'},
    height=500
)

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(template='plotly_white')

fig.show()
fig.write_image("../outputs/figures/04_year_over_year_changes.png", scale=2)

# Highlight McLaren's changes
print("\n🎯 McLaren Year-over-Year Changes:")
mclaren_yoy = yoy_df[yoy_df['Team'] == 'MCLAREN'].sort_values('Season_Start')
print(mclaren_yoy[['Year', 'Change']].to_string(index=False))


VISUALIZATION 4: YEAR-OVER-YEAR CHANGES



🎯 McLaren Year-over-Year Changes:
     Year  Change
2017-2018    32.0
2018-2019    83.0
2019-2020    57.0
2020-2021    73.0
2021-2022  -116.0
2022-2023   143.0
2023-2024   364.0


# ============================================================================
# VISUALIZATION 5: PLACEBO UNITS
# ============================================================================


In [7]:
print("\n" + "="*80)
print("VISUALIZATION 5: PLACEBO UNITS (SAUBER & HAAS)")
print("="*80)

fig = go.Figure()

# McLaren (reference)
fig.add_trace(go.Scatter(
    x=treated_df['season'],
    y=treated_df['points'],
    mode='lines+markers',
    name='McLaren (Treated)',
    line=dict(color='#FF8700', width=4),
    marker=dict(size=10)
))

# Sauber (primary placebo)
fig.add_trace(go.Scatter(
    x=placebo_sauber_df['season'],
    y=placebo_sauber_df['points'],
    mode='lines+markers',
    name='Sauber (Primary Placebo)',
    line=dict(color='#900000', width=3, dash='dash'),
    marker=dict(size=8)
))

# Haas (secondary placebo)
fig.add_trace(go.Scatter(
    x=placebo_haas_df['season'],
    y=placebo_haas_df['points'],
    mode='lines+markers',
    name='Haas (Secondary Placebo)',
    line=dict(color='#FFFFFF', width=3, dash='dot'),
    marker=dict(size=8, line=dict(color='black', width=1))
))

fig.add_vline(x=2021.5, line_dash="dash", line_color="red")

fig.update_layout(
    title='Placebo Units vs McLaren: Expected No Treatment Effect',
    xaxis_title='Season',
    yaxis_title='Constructor Points',
    height=600,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()
fig.write_image("../outputs/figures/05_placebo_units.png", scale=2)


VISUALIZATION 5: PLACEBO UNITS (SAUBER & HAAS)


# ============================================================================
# VISUALIZATION 6: RACE-LEVEL ANALYSIS
# ============================================================================


In [8]:
if race_df is not None:
    print("\n" + "="*80)
    print("VISUALIZATION 6: RACE-LEVEL WITHIN-SEASON DYNAMICS")
    print("="*80)
    
    # Focus on 2024 (McLaren's championship surge)
    race_2024 = race_df[race_df['season'] == 2024]
    
    # Get teams to plot
    teams_race = ['MCLAREN', 'FERRARI', 'RED BULL', 'MERCEDES']
    
    fig = go.Figure()
    
    race_colors = {
        'MCLAREN': '#FF8700',
        'FERRARI': '#DC0000',
        'RED BULL': '#1E41FF',
        'MERCEDES': '#00D2BE'
    }
    
    for team in teams_race:
        team_data = race_2024[race_2024['constructorName'] == team].sort_values('round')
        
        if len(team_data) > 0:
            fig.add_trace(go.Scatter(
                x=team_data['round'],
                y=team_data['cumulative_points'],
                mode='lines+markers',
                name=team,
                line=dict(color=race_colors.get(team, '#808080'), width=3),
                marker=dict(size=6)
            ))
    
    fig.update_layout(
        title='2024 Season: Cumulative Points by Race',
        xaxis_title='Race Number',
        yaxis_title='Cumulative Points',
        height=600,
        template='plotly_white',
        hovermode='x unified'
    )
    
    fig.show()
    fig.write_image("../outputs/figures/06_race_by_race_team_points_2024.png", scale=2)
    
    # When did McLaren overtake Ferrari?
    mclaren_2024 = race_2024[race_2024['constructorName'] == 'MCLAREN'].sort_values('round')
    ferrari_2024 = race_2024[race_2024['constructorName'] == 'FERRARI'].sort_values('round')
    
    if len(mclaren_2024) > 0 and len(ferrari_2024) > 0:
        merged = pd.merge(
            mclaren_2024[['round', 'cumulative_points']],
            ferrari_2024[['round', 'cumulative_points']],
            on='round',
            suffixes=('_mclaren', '_ferrari')
        )
        
        merged['mclaren_ahead'] = merged['cumulative_points_mclaren'] > merged['cumulative_points_ferrari']
        
        if merged['mclaren_ahead'].any():
            first_ahead = merged[merged['mclaren_ahead']].iloc[0]
            print(f"\n🏎️ McLaren first overtook Ferrari after Race {int(first_ahead['round'])}")



VISUALIZATION 6: RACE-LEVEL WITHIN-SEASON DYNAMICS



🏎️ McLaren first overtook Ferrari after Race 13


# ============================================================================
# KEY INSIGHTS SUMMARY
# ============================================================================


In [9]:
print("\n" + "="*80)
print("KEY INSIGHTS FOR SCM DESIGN")
print("="*80)

print("\n1. PRE-TREATMENT FIT:")
print(f"   McLaren 2017-2020 average: {pre_treatment[pre_treatment['constructorName'] == 'MCLAREN']['points'].mean():.1f} points")
print(f"   Alpine 2017-2020 average: {pre_treatment[pre_treatment['constructorName'] == 'ALPINE']['points'].mean():.1f} points")
print(f"   Sauber 2017-2020 average: {pre_treatment[pre_treatment['constructorName'] == 'SAUBER']['points'].mean():.1f} points")
print(f"   Haas 2017-2020 average: {pre_treatment[pre_treatment['constructorName'] == 'HAAS']['points'].mean():.1f} points")

print("\n2. TREATMENT EFFECT:")
mclaren_change = comparison_df[comparison_df['Team'] == 'MCLAREN'].iloc[0]
print(f"   McLaren change: +{mclaren_change['Change']:.1f} points ({mclaren_change['Change %']:.1f}% increase)")
print(f"   Largest donor change: {comparison_df.iloc[1]['Team']}: {comparison_df.iloc[1]['Change']:.1f} points")

print("\n3. PRE-TREATMENT TRENDS:")
mclaren_pre_trend = pre_treatment[pre_treatment['constructorName'] == 'MCLAREN'].sort_values('season')
print(f"   McLaren 2017: {mclaren_pre_trend.iloc[0]['points']:.0f} → 2020: {mclaren_pre_trend.iloc[-1]['points']:.0f} (strong upward trend)")
print("   ⚠️ Challenge: McLaren has non-parallel pre-treatment trend")
print("   Solution: SCM weights will need to match this upward trajectory")

print("\n4. PLACEBO EXPECTATIONS:")
sauber_change = comparison_df[comparison_df['Team'] == 'SAUBER'].iloc[0]
haas_change = comparison_df[comparison_df['Team'] == 'HAAS'].iloc[0]
print(f"   Sauber change: {sauber_change['Change']:.1f} points (expected: small/zero)")
print(f"   Haas change: {haas_change['Change']:.1f} points (2021 tank confounds interpretation)")

print("\n5. DONOR POOL QUALITY:")
print("   ✅ Alpine: Similar level to McLaren, good pre-treatment match")
print("   ⚠️ Sauber: Much weaker, but useful for low-end matching")
print("   ⚠️ Haas: Very volatile, but covers McLaren's 2017 low point")
print("   Decision: Use all three, let SCM optimization determine weights")

print("\n" + "="*80)
print("EXPLORATORY ANALYSIS COMPLETE ✅")
print("="*80)


KEY INSIGHTS FOR SCM DESIGN

1. PRE-TREATMENT FIT:
   McLaren 2017-2020 average: 109.8 points
   Alpine 2017-2020 average: 112.8 points
   Sauber 2017-2020 average: 29.5 points
   Haas 2017-2020 average: 42.8 points

2. TREATMENT EFFECT:
   McLaren change: +265.9 points (242.3% increase)
   Largest donor change: ALPINE: 6.6 points

3. PRE-TREATMENT TRENDS:
   McLaren 2017: 30 → 2020: 202 (strong upward trend)
   ⚠️ Challenge: McLaren has non-parallel pre-treatment trend
   Solution: SCM weights will need to match this upward trajectory

4. PLACEBO EXPECTATIONS:
   Sauber change: -4.5 points (expected: small/zero)
   Haas change: -7.1 points (2021 tank confounds interpretation)

5. DONOR POOL QUALITY:
   ✅ Alpine: Similar level to McLaren, good pre-treatment match
   ⚠️ Sauber: Much weaker, but useful for low-end matching
   ⚠️ Haas: Very volatile, but covers McLaren's 2017 low point
   Decision: Use all three, let SCM optimization determine weights

EXPLORATORY ANALYSIS COMPLETE ✅
